# Risk Processing Steps


This Jupyter notebook contains all the calculation steps for the market mechanism, which are used for statistical analysis. 

In [1]:
# --------- Necessary import of data preprocessing

import sys, glob, json, re
from pathlib import Path
import numpy as np
import pandas as pd
import neurokit2 as nk
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import math
import statsmodels.formula.api as smf
import seaborn as sns
import os
import itertools
import warnings
warnings.filterwarnings("ignore")

In [2]:
# --- Paths & Load data
ROOT_DIR = "/Users/johannanagel/Library/CloudStorage/GoogleDrive-nagel.johanna1@gmail.com/Meine Ablage/Masterarbeit_EDA_ECG_Signals/Kopie von Findec ECG_EDA_EEG Data all sessions"
OUTDIR = Path("/Users/johannanagel/Masterarbeit_Programming/output_postfeedback")
ANL_OUT = OUTDIR / "analysis_ready_primary_5s.csv"
RESULTS = Path("/Users/johannanagel/Masterarbeit_Programming/output_postfeedback/Main_Part/DataFrames")
os.makedirs(RESULTS, exist_ok = True)

In [3]:
def find_particpants_and_sessions(Path):
    """ 
    Finding and retrieving session and participants of the file 
    Params: 
        Path (string): path to the file (e.g analysis_ready_primary_5s.csv)
    Returns:
        df_participants (DataFrame): DataFrame of participants and sessions
    """
    df = pd.read_csv(Path, low_memory=False)
    if df is None:
        raise ValueError("[Error] No file")
    cols = ["session", "participant"]
    #sorting and dropping duplicates
    df_participant = (df[cols].sort_values(cols).drop_duplicates(subset=["session", "participant"], keep="first"))
    return df_participant



def read_otree_file(Path, df_participant):
    """
    Reading otree files of all sessions, 
    Retrieving data of analysis ready participants.
       -  Extracting participants who are excluded and included
       -  Retrieving demographics of each participant

    Params:
        Path (string): path to the o-tree file 
        df_participants (DataFrame): Dataframe participants and sessions
    Return:
        participants_included (DataFrame): Dataframe with all relevant trading informations of each (included) participant
        participants_excluded (List): All participants who are excluded
        participants_demographics (DataFrame): Dataframe with all relevant demographics of each (included) participant

    """
    sessions = set(df_participant['session'].astype(str).unique())
    participants = set(df_participant['participant'].astype(str).unique())
    round_number = list(range(1,41))
    rows = []
    rows_demographics = []
    particiants_excluded = []
    session_folders_names = [name for name in os.listdir(Path) if os.path.isdir(os.path.join(Path, name)) and name in sessions]

    if session_folders_names is None:
        raise ValueError("[Error] No session names")

    for session_names in session_folders_names:
        session_path = os.path.join(Path,session_names)
        session_files = os.listdir(session_path)
        otree_file = [file for file in session_files if file.startswith('all_apps_wide-')][0]
        path = os.path.join(session_path,otree_file)
        otree_log = pd.read_csv(path)
        for index, row in otree_log.iterrows():
            #checking if participant of the otree file in analysis_ready
             if row['participant.code'] in participants:
                # Extracting demogrphics and trading variables
                #print(f"participant {row['participant.code']} included")

                trading_experience = 'introduction.1.player.trading_experience'
                age = 'introduction.1.player.age'
                gender = 'introduction.1.player.gender'
                occupation = 'introduction.1.player.occupation'
                fatigue_state = 'introduction.1.player.fatigue_state'
                final_wallet = 'participant.wallet'
                final_portfolio = 'participant.portfolio'
                new_row_demograpghics = {"session": session_names, "participant": row['participant.code'], "trading_experience": row[trading_experience], "age": row[age],"gender": row[gender],"occupation": row[occupation],"fatigue_state": row[fatigue_state], "final_portfolio": row[final_portfolio],"final_wallet": row[final_wallet] }
                rows_demographics.append(new_row_demograpghics)
                for i in round_number:
                    dividend = f'stockmarket.{i}.player.dividend'
                    interest = f'stockmarket.{i}.player.interest'
                    cash = f'stockmarket.{i}.player.cash'
                    stocks = f'stockmarket.{i}.player.stocks'
                    new_row = {"session": session_names, "participant": row['participant.code'], "round": i, "dividend": row[dividend], "interest": row[interest], "cash": row[cash], "stocks": row[stocks], "final_portfolio": row[final_portfolio],"final_wallet": row[final_wallet] }
                    rows.append(new_row)
             else: 
                 #print(f"participant {row['participant.code']} excluded")
                 particiants_excluded.append(row['participant.code'])

    participants_included = pd.DataFrame(rows)
    participants_demographics = pd.DataFrame(rows_demographics)

    print(f"Number of sessions: {len(sessions)}")
    print(f"Number of excluded participants: {len(particiants_excluded)}")
    print(f"Number of included participants: {participants_included["participant"].drop_duplicates().shape[0]}")

    return participants_included, participants_included, participants_demographics



def left_join(ANL, otree):
    """
    Method for merging two DataFrames (method = left)
    Params:
        ANL (DataFrame): Dataframe with EDA signals, trading variables
        otree (DataFrame): Dataframe with otree informations
    Return:
        df (Dataframe): Merged (ANL + otree)
    """
    return pd.merge(ANL, otree, how="left", on=["session", "participant", "round"])

try: 
    df_participant = find_particpants_and_sessions(ANL_OUT)

    otree_all, particiants_excluded, participants_demographics = read_otree_file(ROOT_DIR, df_participant)

    df = pd.read_csv(ANL_OUT, low_memory=False)
    df = left_join(df, otree_all)

    df = df[df["is_valid"]]

    test = df[['session', 'participant', 'round', 'dividend', 'interest', 'cash', 'stocks']]

    #checking for nan values
    print(f"\nChecking for nan values:\n{test.isna().sum()}")
except ValueError as err:
    print(f"[Error] in finding and retrieving data: {err}")


Number of sessions: 25
Number of excluded participants: 62
Number of included participants: 125

Checking for nan values:
session        0
participant    0
round          0
dividend       0
interest       0
cash           0
stocks         0
dtype: int64


In [ ]:
def remove_perticpants_with_small_round_sizes(df, participants_demographics):
    """
    Checking if each participant has enough rounds participation
    Removing participants who are below the defined treshhold
    Params: 
        df (Dataframe): Dataframe - general (EDA signals, round, transaction, price)
        participants_demographics (DataFrame): Dataframe -  demographics
    Return: 
        df_filtered (Dataframe): Data of participants who fullfill the condition
        df_filtered_demographics: (DataFrame): Data (demographics)  of participants who fullfill the condition
    """
    
    if df is None:
        raise ValueError("No dataframe")

    # Group dataframe with participant, Checking number of unique round_numbers
    rounds_per_participant = df.groupby("participant")["round"].nunique()
    print(f"Number of rounds per participant:\n{rounds_per_participant}")
    print("\n🖊️ Describe number of rounds per participants: ")
    print(rounds_per_participant.describe())

    #Defining a treshhold for the minimum number of rounds
    treshhold= round(rounds_per_participant.mean(),2)
    
    #Validate if each participant fullfill this condition (roundnumber >= reshhold)
    valid_participants = rounds_per_participant[rounds_per_participant >=treshhold].index
    df_filtered = df[df["participant"].isin(valid_participants)]

    #Retrieve all participants who fulfill this condition
    df_filtered_demographics = participants_demographics[participants_demographics["participant"].isin(valid_participants)]
    print(f"\nRemoved number of participants: {len(rounds_per_participant)- len(valid_participants)}")
    print(f"Keep number of participants: {len(valid_participants)} von {len(rounds_per_participant)}")


    return df_filtered, df_filtered_demographics


def remove_sessions_without_40_round_of_sessions(df):
    """
    Removing sessions without 40 trading rounds 
    """

    if df is None:
        raise ValueError("No dataframe") 

    rounds_per_session = df.groupby("session")["round"].nunique()
    incomplete_sessions = rounds_per_session[rounds_per_session != 40]  
    print("=== Check: Number of Rounds per Session ===")
    print(rounds_per_session.sort_values())

    if incomplete_sessions.empty:
        print("\n✅ All sessions have 40 rounds.")
    else:
        print("\n❌ Following sessions do not have 40 rounds:")
        print(incomplete_sessions)

try:    
    df, participants_demographics = remove_perticpants_with_small_round_sizes(df, participants_demographics)
    remove_sessions_without_40_round_of_sessions(df)
except ValueError as err:
    print(f"[Error]: {err}")


Number of rounds per participant:
participant
07cwkx8l     1
0b8254y9    40
0k5o6enp    40
0nppklhu    14
14qogk6l    40
            ..
ydjo4io7    32
ypnhlu5o    40
zd8tyma1    38
zgwrjgxr    37
zpxjh4d7    35
Name: round, Length: 125, dtype: int64

🖊️ Describe number of rounds per participants: 
count    125.000000
mean      30.536000
std       13.933913
min        1.000000
25%       26.000000
50%       39.000000
75%       40.000000
max       40.000000
Name: round, dtype: float64

Removed number of participants: 36
Keep number of participants: 89 von 125
=== Check: Number of Rounds per Session ===
session
02-07-2024_14.15 (pre LSL)    40
28-08-2024_15.15              40
28-05-2024_17.15 (pre LSL)    40
28-05-2024_14.30 (pre LSL)    40
27_11_2024                    40
27-08-2024_13.45              40
21_11_2024 (no fMRI)          40
20-08-2024_16.15(no fMRI)     40
20-08-2024_13.45(no fMRI)     40
28_11_2024                    40
19_11_2024                    40
14-08-2024_15.15      

### Performance

1. Calculate the final performance of each participant by calculating the final earnings:
<br>    - final_money = final_wallet + final_portfolio * 14 
2. Categorize the final perfomance into 3 similar size categories by using pcut (global)
3. Calculating the perfomance for each session (by calculating the mean final earnings per session and splitting with pcut in 3 categories


In [ ]:

try:
    #Calculating final earnings for each participant
    participants_demographics["final_money"] = participants_demographics["final_wallet"] + participants_demographics["final_portfolio"] * 14

    participants_demographics[["final_money", "final_wallet", "final_portfolio", "gender"]].sort_values("final_money", ascending= False)

    # which gender made the most money
    print("Which gender made the most money: ")
    print(participants_demographics.groupby("gender")["final_money"].median())


    #Classify the final earnings of each participant (global)
    participants_demographics["performance_classification"] = pd.qcut(participants_demographics["final_money"],q=3, labels = ["Low Performer", "Medium Performer", "High Performer"])

    #sorting values and return top 5 earners
    top_5_best_earners = participants_demographics.sort_values("final_money", ascending= False).head(10)

    print(f"\n🏆 Top 5 best earners: \n")
    for idx, row in top_5_best_earners.iterrows():
        print(f"session: {row["session"]}, participant: {row["participant"]},  final earnings: {row["final_money"]}" )
    bottom_5_earners = participants_demographics.sort_values("final_money", ascending= False).tail(10)
    print(f"\n👎 Bottom 5 best earners: \n")
    for idx, row in bottom_5_earners.iterrows():
        print(f"session: {row["session"]}, participant: {row["participant"]},  final earnings: {row["final_money"]}" )
    
    # Classify performance session-wise
    participants_demographics["performance_classification_session_wise"] = None
    for session, group in participants_demographics.groupby("session")["final_money"]:
        participants_demographics.loc[group.index, "performance_classification_session_wise"] = pd.qcut(group,q=3, labels = ["Low Performer", "Medium Performer", "High Performer"])


    #Calculating the median of the final earnings per session: 
    session_median_earnings = participants_demographics.groupby("session")["final_money"].median().round(2).rename("final_earnings_mean")
    # Classify the median earnings (terciles)
    session_classification = pd.qcut(session_median_earnings,q=3, labels = ["Low Performer", "Medium Performer", "High Performer"]).rename("performance_classification_session")
    session_info = pd.concat([session_median_earnings, session_classification], axis=1)
    participants_demographics = pd.merge(participants_demographics, session_info, on = ["session"], how = "left" )





    # save participants_demographics
    participants_demographics.to_csv(os.path.join(RESULTS, "demographics_participant.csv"), index=False)  

    ### 
    values = participants_demographics.performance_classification.unique()
    classification_performers = participants_demographics.groupby("performance_classification")["gender"].value_counts()

    print("\n🔍 Number of participants per gender per performance-categories\n",classification_performers )

except ValueError as err:
    print(err)

Which gender made the most money: 
gender
Female    305.985
Male      321.240
Name: final_money, dtype: float64

🏆 Top 5 best earners: 

session: 13_11_2024, participant: 4x9jc1cv,  final earnings: 917.12
session: 28-05-2024_14.30 (pre LSL), participant: dzw9hmec,  final earnings: 649.56
session: 20-08-2024_16.15(no fMRI), participant: r4pjw3xb,  final earnings: 588.78
session: 27-08-2024_13.45, participant: i338vbgf,  final earnings: 557.4200000000001
session: 14-08-2024_15.15, participant: 1w4qroin,  final earnings: 534.4100000000001
session: 27-08-2024_13.45, participant: hedk3opj,  final earnings: 529.47
session: 13_11_2024, participant: owih6fgs,  final earnings: 490.32
session: 07_11_2024, participant: um7lwz3z,  final earnings: 476.33
session: 20-08-2024_16.15(no fMRI), participant: nz59smtl,  final earnings: 441.11
session: 20-08-2024_13.45(no fMRI), participant: 4m90nz99,  final earnings: 424.53

👎 Bottom 5 best earners: 

session: 02-07-2024_16.30 (pre LSL), participant: 6a1g

In [6]:
participants_demographics[["session","participant", "final_money", "performance_classification", "performance_classification_session_wise", "final_earnings_mean"]]

,session,participant,final_money,performance_classification,performance_classification_session_wise,final_earnings_mean
0,28-05-2024_14.30 (pre LSL),uspyk0br,243.41,Low Performer,Low Performer,322.42
1,28-05-2024_14.30 (pre LSL),dzw9hmec,649.56,High Performer,High Performer,322.42
2,28-05-2024_14.30 (pre LSL),3uuuvlfn,266.97,Low Performer,Low Performer,322.42
3,28-05-2024_14.30 (pre LSL),3td05myb,377.87,High Performer,High Performer,322.42
4,14_11_2024,0b8254y9,169.65,Low Performer,Low Performer,229.92
...,...,...,...,...,...,...
84,03-07-2024_15.15 (pre LSL),rw33oedj,151.75,Low Performer,Low Performer,300.97
85,03-07-2024_15.15 (pre LSL),k7rj36eg,390.58,High Performer,High Performer,300.97
86,03-07-2024_15.15 (pre LSL),sbnjsla1,247.36,Low Performer,Medium Performer,300.97
87,19_11_2024,r6mbt4v9,250.31,Low Performer,Low Performer,322.49


In [7]:

# Define subsets of columns
cols_calc = ["session", "round", "price"]
cols_trans = ["session", "round", "participant", "price", "transaction"]
cols_participant = ["session", "round", "participant"]


# Price DataFrame: keeps unique session, round
# if multiple prices per round exist, take the first after sorting
df_calc = (
    df[cols_calc]
    .sort_values(["session", "round"])
    .drop_duplicates(subset=["session", "round"], keep="first")
)

# Transaction DataFrame (keeps unique individual transactions)
df_transaction = (
    df[cols_trans]
    .drop_duplicates()
)

sell_counts = (
    df_transaction.assign(is_sold=lambda x: x["transaction"].eq("sell"))
    .groupby(["session", "round"], as_index=False)["is_sold"].sum()
    .rename(columns={"is_sold": "sell_count"})
    .astype({"sell_count": int})
)

print(sell_counts["sell_count"].value_counts())

# Participant DataFrame 
df_participants = (df[cols_participant].sort_values(["session", "round", "participant"]).drop_duplicates())
count_participants_round = (
    df_participants.groupby(["session", "round"])["participant"]
      .nunique()
      .reset_index(name="n_participants")
)


# Merge sell counts back into both DataFrames
df_calc = df_calc.merge(sell_counts, on=["session", "round"], how="left").fillna({"sell_count": 0}).astype({"sell_count": int})
df_calc = df_calc.merge(count_participants_round, on=["session", "round"], how="left").fillna({"n_participants": 0}).astype({"n_participants": int})
#calc average number of trades per session
avg_number_of_trades_per_session = round((df_calc.groupby(["session"])["sell_count"].agg("mean")).reset_index(name="avg_trades"),2)

print(f"\nAverage number of trades per round per session: \n{avg_number_of_trades_per_session}")
df_calc = df_calc.merge(avg_number_of_trades_per_session, on=["session"],how="left")




sell_count
0    388
1    386
2     65
3      1
Name: count, dtype: int64

Average number of trades per round per session: 
                       session  avg_trades
0   02-07-2024_14.15 (pre LSL)        0.65
1   02-07-2024_16.30 (pre LSL)        0.80
2   03-07-2024_15.15 (pre LSL)        0.72
3             06-08-2024_13.15        0.82
4                   07_11_2024        0.75
5                   12_11_2024        0.80
6   13-08-2024_13.45 (No fMRI)        0.25
7                   13_11_2024        0.80
8             14-08-2024_15.15        0.75
9                   14_11_2024        0.45
10                  19_11_2024        0.22
11   20-08-2024_13.45(no fMRI)        0.60
12   20-08-2024_16.15(no fMRI)        0.57
13       21_11_2024 (no fMRI)         0.28
14            27-08-2024_13.45        0.68
15                  27_11_2024        0.55
16  28-05-2024_14.30 (pre LSL)        0.88
17  28-05-2024_17.15 (pre LSL)        0.35
18            28-08-2024_15.15        0.48
19               

In [ ]:
# copy eda variables for its own dataframe
df_eda = df[["session", "participant", "round", "price", "baseline_corrected",  "scl_mean", "scl_std", "scr_amplitude", "scr_auc", "scr_count", "scr_per_sec", "signal_range", "signal_std", "window_type"]].copy().sort_values(["session", "participant", "round", "price"])

## Fundamental Value

This section calculates the FV by 3 different research papers:
- Smith et. al (2014)
- Giusti (2016) (variables are written wrong -> here Guisty)
- Kirchler (2012)

In [9]:
def calculation_Fundamenetal_Value_by_Smith(expected_dividend = 0.7, interest_rate = 0.01):
    """Calculating Fundamental Value by Smith"""
    FV_Smith = (expected_dividend/interest_rate)
    return FV_Smith
    
def Fundamental_Value_by_Guisty( actual_round, total_round, expected_dividend, interest_rate, fixed_buyout = 14):
    """Calculating Fundamental Value by Guisty, where the fixed_buyout = 14"""
    k = total_round - actual_round + 1
    sum_fv = 0
    term_fixed_buy_out = fixed_buyout * ((1 + interest_rate)**(-(total_round-actual_round +1)))
    for h in range(1,k+1):
        sum_fv += ((1+interest_rate)**(-h)) 
    return round((expected_dividend * sum_fv) + term_fixed_buy_out,2)



def calculating_Fundamental_Value_by_Kirchler(remaining_periods, expected_dividend, terminal_value=14):
    """Calculating Fundamental Value by Kirchler, where the terminal_value = 14"""
    return round(expected_dividend * remaining_periods + terminal_value,2)


def calculation_Fundamental_Values_per_each_period(total_round, expected_dividend, interest_rate):
    """method for calling methods of FV, and adding them to the dataframe """
    df_fv = pd.DataFrame({
        "round": np.arange(1, 41)})
    df_fv["remaining_rounds"] = total_round - df_fv["round"]
    df_fv["FV_SMITH"] = np.nan

    df_fv["FV_GUISTY"] = np.nan
    df_fv["FV_KIRCHLER"] = np.nan

    for idx, row in df_fv.iterrows():
        
        remaining_rounds = int(row["remaining_rounds"])
        actual_round = int(row["round"])
        df_fv.at[idx, "FV_SMITH"] = calculation_Fundamenetal_Value_by_Smith()
        df_fv.at[idx, "FV_GUISTY"] = Fundamental_Value_by_Guisty( actual_round, total_round, expected_dividend, interest_rate)
        df_fv.at[idx, "FV_KIRCHLER"] = calculating_Fundamental_Value_by_Kirchler(remaining_rounds, expected_dividend)
    
    return df_fv



df_fv = calculation_Fundamental_Values_per_each_period(40, 0.7, 0.01)
df_calc = pd.merge(df_calc, df_fv, on="round", how="left")
print("Checking if there are any nan values:")
if df_calc.isnull().values.any():
    print("❌", df_calc.isnull().values.any())
else:
    print("    ✅ No nan values in Dataframe")




Checking if there are any nan values:
    ✅ No nan values in Dataframe


## Cash/ Asset Ratio

- Calculating the Cash/Assest ratio by using the FV by Kirchler, Guisti, and Smith
- C/A - Ratio = Cash/ (FV * Number of stocks)

In [10]:
df_cashA = df[["session", "participant", "round", "cash", "stocks"]].drop_duplicates().copy()
df_fundCashA = df_calc[["session", "round", "price", "FV_KIRCHLER", "FV_SMITH", "FV_GUISTY"]].drop_duplicates().copy()

df_cashA = pd.merge(df_cashA, df_fundCashA, on=["session", "round"], how="left")

def cash_asset_ratio(df):
    """
    Method for calculating the Cash/Asset Ratio
    Params:
        df (DataFrame): All necessary information which are needed for calc C/A ratio
    Return:
        df_cashA (Dataframe): Includes the df and all informations regarding the C/A ratio
    """
    if df is None:
        raise ValueError("No Dataframe available for calculating C/A ratio")

    #calc number of stocks
    number_of_stucks = (df.groupby(["session", "round"])["stocks"].sum().reset_index(name="n_stocks"))
    print(number_of_stucks)
    #calc cash per round of each session
    cash_per_round = (df.groupby(["session", "round"])["cash"].sum().reset_index(name="cash_per_round").round(2))
    temp_df = pd.merge(number_of_stucks, cash_per_round,on=["session", "round"],  how="left")

    df = pd.merge(df,temp_df, on = ["session", "round"], how= "left")
    df = df.sort_values(["session", "round"])
    #calc C/A ratio for each fundamenttal Value
    df["cash_asset_ratio_KIRCHLER"] = round(df["cash_per_round"] / (df["n_stocks"] * df["FV_KIRCHLER"]),2)
    df["cash_asset_ratio_GUISTY"] = round(df["cash_per_round"] / (df["n_stocks"] * df["FV_GUISTY"]),2)
    df["cash_asset_ratio_SMITH"] = round(df["cash_per_round"] / (df["n_stocks"] * df["FV_SMITH"]),2)
    return df

try:    
    df_cashA = cash_asset_ratio(df_cashA)
    #calculating the man ca ration for each FV over all sessions
    df_cashA_mean = df_cashA[["session", "round", "price","cash_asset_ratio_KIRCHLER", "cash_asset_ratio_SMITH", "cash_asset_ratio_GUISTY"]].drop_duplicates()
    mean_ratios = df_cashA_mean.groupby("round")[["cash_asset_ratio_KIRCHLER", "cash_asset_ratio_SMITH", "cash_asset_ratio_GUISTY"]].mean().round(2).reset_index()
    mean_ratios = mean_ratios.rename(columns={col: f"mean_{col}" for col in mean_ratios.columns if col != "round"})

    #merge the mean to original data
    df_cashA = pd.merge(df_cashA, mean_ratios, on= ["round"], how='left')
    #Merging df_cashA to calculation dataframe
    df_calc = pd.merge(df_calc, df_cashA, on= ["session", "round", "price", "FV_GUISTY", "FV_KIRCHLER", "FV_SMITH"], how="left")
except ValueError as err:
    print("[Error] by calculating C/A ratio: ", err)

                        session  round  n_stocks
0    02-07-2024_14.15 (pre LSL)      1        25
1    02-07-2024_14.15 (pre LSL)      2        25
2    02-07-2024_14.15 (pre LSL)      3        25
3    02-07-2024_14.15 (pre LSL)      4        25
4    02-07-2024_14.15 (pre LSL)      5        24
..                          ...    ...       ...
835  29-05-2024_15.30 (pre LSL)     36        31
836  29-05-2024_15.30 (pre LSL)     37        30
837  29-05-2024_15.30 (pre LSL)     38        30
838  29-05-2024_15.30 (pre LSL)     39        30
839  29-05-2024_15.30 (pre LSL)     40        30

[840 rows x 3 columns]


## Relative Deviation and Relative Absolute Deviation

Based on the FV by researcher Smith, Kirchler, and Guisty, calculating the relative and relative absolute deviation of the market price and the FV.

### Overall RD, RAD

In [ ]:
#Calc RD, RAD for SMITH, GIUSTI and KIRCHLER (general Deviation)

def calculation_average_FV(df_group, total_round):
    """
    Calculating the average FV of a group (= session)
    """
    if df_group is  None:
        raise ValueError("No Dataframe for calculating average FV")
    if "FV_GUISTY" not in df_group.columns:
        raise ValueError("No FV_GUISTY in df")
    return round(sum(df_group["FV_GUISTY"])/total_round,2)

def calculation_RD_Guisty(df_group, total_round, FV_median):
    """ Calculating Relative Deviation for the whole session (result = one value)"""

    if df_group is  None:
        raise ValueError("No Dataframe for calculating average FV")
    if  total_round is None or  FV_median is None:
        raise ValueError("No total_round, FV_median  for calculating average FV")
    if "price" not in df_group.columns and "FV_GUISTY" not in df_group.columns :
        raise ValueError("No price or FV in df")

    rd_interim = 0
    for t in range(1, total_round + 1):
        price = df_group.iloc[t-1]["price"]
        FV_t  = df_group.iloc[t-1]["FV_GUISTY"]
        rd_interim += ((price - FV_t)/FV_median)
    return round(rd_interim/total_round,2)

def calculation_RAD_Guisty(df_group, total_round, FV_median):
    """ Calculating Relative Absolute Deviation for the whole session (result = one value)"""

    if df_group is  None:
        raise ValueError("No Dataframe for calculating average FV")
    if "price" not in df_group.columns or "FV_GUISTY" not in df_group.columns or total_round is None or  FV_median is None :
        raise ValueError("Necessary variabel not in df")

    rad_interim = 0
    for t in range(1, total_round + 1):
        price = df_group.iloc[t-1]["price"]
        FV_t  = df_group.iloc[t-1]["FV_GUISTY"]
        rad_interim += ((abs(price - FV_t))/FV_median)
    return round(rad_interim/total_round,2)

def calculation_RD_SMITH(df_group, total_round, FV_median, debug):
    """ Calculating Relative Deviation for the whole session (result = one value)"""

    if df_group is  None:
        raise ValueError("No Dataframe for calculating average FV")
    if "price" not in df_group.columns or "FV_SMITH" not in df_group.columns or total_round is None  or "sell_count" not in df_group.columns :
        raise ValueError("Necessary variabel not in df")

    rd_interim = 0
    sum_vt = 0
    for t in range(1, total_round + 1):
        if debug:
            price = df_group.iloc[t-1]["price"]
            print("price:", price)
            FV_t  = df_group.iloc[t-1]["FV_SMITH"]
            #v_t number of units traded in each round
            v_t = df_group.iloc[t-1]["sell_count"]
            print("v_t", v_t)
            rd_interim += (v_t* ((price - FV_t)/FV_t))
            sum_vt += v_t
        else:
            price = df_group.iloc[t-1]["price"]
            FV_t  = df_group.iloc[t-1]["FV_SMITH"]
            #v_t number of units traded in each round
            v_t = df_group.iloc[t-1]["sell_count"]
            rd_interim += (v_t* ((price - FV_t)/FV_t))
            sum_vt += v_t

    return round(rd_interim/sum_vt,2)


def calculation_RAD_SMITH(df_group, total_round, debug):
    """ Calculating Relative Deviation for the whole session (result = one value)"""

    if df_group is  None:
        raise ValueError("No Dataframe for calculating average FV")
    if "price" not in df_group.columns or "FV_SMITH" not in df_group.columns or total_round is None or "sell_count" not in df_group.columns:
        raise ValueError("Necessary variabel not in df")

    rad_interim = 0
    sum_vt = 0
    for t in range(1, total_round + 1):
        price = df_group.iloc[t-1]["price"]
        FV_t  = df_group.iloc[t-1]["FV_SMITH"]
        #v_t number of units traded in each round
        v_t = df_group.iloc[t-1]["sell_count"]
        rad_interim += (v_t* ((abs(price - FV_t))/FV_t))
        sum_vt += v_t

    if debug == True:
        print(rad_interim)
        print(sum_vt)
        print(rad_interim/sum_vt)
    return round(rad_interim/sum_vt,2)

def share_turnover(df_group, total_round, number_outstanding_shares):
    """ Calculating the number of traded shares / number_outstanding_shares """
    if df_group is  None:
        raise ValueError("No Dataframe for calculating average FV")

    number_outstanding_shares = df_group.iloc[0]["n_participants"]* 5
    sum_shares = sum(df_group["sell_count"])
    return round(sum_shares/number_outstanding_shares,2)

def calc_RAD_KIRCHLER(df_group, total_round, FV_mean_Kirchler):
    """ Calculating Relative Deviation for the whole session (result = one value)"""

    if df_group is  None:
        raise ValueError("No Dataframe for calculating average FV")
    if "price" not in df_group.columns or "FV_KIRCHLER" not in df_group.columns or total_round is None or  FV_mean_Kirchler is None :
        raise ValueError("Necessary variabel not in df")
    rad_interim = 0
    for t in range(1, total_round + 1):
        price = df_group.iloc[t-1]["price"]
        FV_t  = df_group.iloc[t-1]["FV_KIRCHLER"]
        rad_interim += (((abs(price - FV_t))/(abs(FV_mean_Kirchler))))
    #print("RAD: ", round(((1/total_round)* rad_interim),2))
    return round(((1/total_round)* rad_interim),2)

def calc_RD_KIRCHLER(df_group, total_round, FV_mean_Kirchler):
    """ Calculating Relative Deviation for the whole session (result = one value)"""

    if df_group is  None:
        raise ValueError("No Dataframe for calculating average FV")
    if "price" not in df_group.columns or "FV_KIRCHLER" not in df_group.columns or total_round is None or  FV_mean_Kirchler is None :
        raise ValueError("Necessary variabel not in df")

    rd_interim = 0
    for t in range(1, total_round + 1):
        price = df_group.iloc[t-1]["price"]
        FV_t  = df_group.iloc[t-1]["FV_KIRCHLER"]
        rd_interim += ((price - FV_t)/(abs(FV_mean_Kirchler)))
    return round(((1/total_round)* rd_interim),2)
        
### Executing RAD, RD methods for an overall value
try: 

    debug = False


    df_fv_values = pd.DataFrame(columns=['session', 'rd_Guisty', 'rad_Guisty', 'rd_Smith', 'rad_Smith', 'share_turnover'])
    df_calc_dev = df_calc[["session", "round", "price", "FV_KIRCHLER","FV_SMITH", "FV_GUISTY", "n_participants","sell_count"]].drop_duplicates().copy()
    #group Dataframe in sessions
    #apply RD, RD methods for the overall measurement
    for session, group in df_calc_dev.groupby("session"):
        group = group.drop_duplicates()

        total_round = 40
        #print(session)
        #if session == "28-05-2024_17.15 (pre LSL)":
            #debug = True
        FV_average_GUISTY = calculation_average_FV(group, total_round)
        #print(f"    average: {FV_average}")
        FV_mean_KIRCHLER = group['FV_KIRCHLER'].mean()
        #print(FV_mean_KIRCHLER)
        rd_GUISTY = calculation_RD_Guisty(group,total_round, FV_average_GUISTY)
        #print(f"    rd Guisty = {rd_GUISTY}")
        rad_GUISTY = calculation_RAD_Guisty(group,total_round, FV_average_GUISTY)
        #print(f"    rad Guisty = {rad_GUISTY}")
        rd_SMITH = calculation_RD_SMITH(group,total_round, FV_average_GUISTY, debug)
        #print(f"    rd SMIRH = {rd_SMITH}")
        #debug = False
        rad_SMITH = calculation_RAD_SMITH(group, total_round, debug)
        #print(f"    rad SMITH = {rad_SMITH}")
        rd_KIRCHLER = calc_RD_KIRCHLER(group, total_round,FV_mean_KIRCHLER)
        #print(f"    rad KIRCHLER = {rad_SMITH}")
        rad_KIRCHLER = calc_RAD_KIRCHLER(group, total_round, FV_mean_KIRCHLER)
        #print(f"    rad KIRCHLER = {rad_SMITH}")
        st = share_turnover(group,total_round, 10)
        #print(f"    share turnover: {st}\n")
        new_row = pd.DataFrame({'session': [session],'av_FV_Guisty':[FV_average_GUISTY],  'rd_Guisty': [rd_GUISTY],'rad_Guisty':[rad_GUISTY] , 'rd_Smith':[rd_SMITH] , 'rad_Smith':[rad_SMITH] ,'rd_Kirchler':[rd_KIRCHLER], 'rad_Kirchler':[rad_KIRCHLER],
                                'share_turnover':[st]  })
        df_fv_values = pd.concat([df_fv_values, new_row], ignore_index=True)

    print(f"Session with max rad_Guisty : \n{df_fv_values['session'].loc[df_fv_values["rad_Guisty"].idxmax()]} \n")
    print(f"Session with min rad_Guisty : \n{df_fv_values['session'].loc[df_fv_values["rad_Guisty"].idxmin()]}\n")        

    print(f"Session with max rad_Smith : \n{df_fv_values['session'].loc[df_fv_values["rad_Smith"].idxmax()]} \n")
    print(f"Session with min rad_Smith : \n{df_fv_values['session'].loc[df_fv_values["rad_Smith"].idxmin()]}") 

    print("\n\n\nChecking nan values")
    if df_fv_values.isnull().values.any():
        print("❌\n",df_fv_values.isna().sum())
        df_fv_values = df_fv_values.fillna(0)
    else:
        print("    ✅ No nan values in Dataframe")

    df_fv_values["Category_SMITH"] = np.where(df_fv_values["rd_Smith"] > 0, "Overpricing",
                    np.where(df_fv_values["rd_Smith"] < 0, "Underpricing", "Fair"))
    df_fv_values["Category_Guisty"] = np.where(df_fv_values["rd_Guisty"] > 0, "Overpricing",
                    np.where(df_fv_values["rd_Guisty"] < 0, "Underpricing", "Fair"))
    df_fv_values["Category_KIRCHLER"] = np.where(df_fv_values["rd_Kirchler"] > 0, "Overpricing",
                    np.where(df_fv_values["rd_Kirchler"] < 0, "Underpricing", "Fair"))
    df_fv_values.head(30)



    df_calc = pd.merge(df_calc, df_fv_values, on= "session", how = "left")

except ValueError as err:
    print(f"[Error] Calculating RD, RAD: {err}")


Session with max rad_Guisty : 
27-08-2024_13.45 

Session with min rad_Guisty : 
29-05-2024_15.30 (pre LSL)

Session with max rad_Smith : 
21_11_2024 (no fMRI)  

Session with min rad_Smith : 
13_11_2024



Checking nan values
    ✅ No nan values in Dataframe


### Calculation of RAD, RD of each round

In this chapter the relative absolute and relative deviation of each round will be calculated.

In [13]:

#using df_price

def calculation_average_FV_Guisty(df_group, total_round=40):
    """ Average FV by Giusti """
    if df_group is None:
        raise ValueError("No Dataframe for calculating average FV")
    #return round(sum(df_group["FV_GUISTY"])/total_round,2)
    #print(len(df_group))
    return round(sum(df_group)/total_round,2)
 
def calc_RD_GUISTY_per_round(df):
    """Calculating Relative Deviation per round """
    if df is None:
        raise ValueError("No Dataframe for calculating average FV")
    if 'price' not in df or "FV_GUISTY" not in df or "avg_FV_GUISTY" not in df :
        raise ValueError("Necessary variables are missing")

    df['rd_GUISTY_per_round'] = round(((df['price'] - df['FV_GUISTY']) / df['avg_FV_GUISTY']),2)
    return df

def calculation_RAD_Guisty_per_round(df):
    """Calculating Relative Absolute Deviation per round """

    if df is None:
        raise ValueError("No Dataframe for calculating average FV")
    if 'price' not in df or "FV_GUISTY" not in df or "avg_FV_GUISTY" not in df :
        raise ValueError("Necessary variables are missing")

    df['rad_GUISTY_per_round'] = round((abs(df['price'] - df['FV_GUISTY']) / df['avg_FV_GUISTY']),2)
    return df

def calc_deviation_Guisty(df):
    """
    Main method for claculating deviations
    Categorizing rd values in overprcing, underpricing, fair
    Overpricing:  RD > 0
    Underpricing: RD < 0
    Fair:         RD =0  
    """

    if df is None:
        raise ValueError("No Dataframe for calculating average FV")
    
    #remove columns if existing
    df = df.drop(columns=[
        "avg_FV_GUISTY",
        "rd_GUISTY_per_round",
        "rad_GUISTY_per_round",
        "Category_Guisty_round",
    ], errors="ignore")

    try: 
        df = df.drop_duplicates()
        f_avg_fv = pd.DataFrame(df.groupby("session")["FV_GUISTY"].apply(calculation_average_FV_Guisty).reset_index(name="avg_FV_GUISTY"))
        df = pd.merge(df, f_avg_fv, on="session", how="left")
        df = calc_RD_GUISTY_per_round(df)
        df = calculation_RAD_Guisty_per_round(df)
        #Categorizing variables
        df["Category_Guisty_round"] = np.where(df["rd_GUISTY_per_round"] > 0, "Overpricing",
                             np.where(df["rd_GUISTY_per_round"] < 0, "Underpricing", "Fair"))
        return df
    except ValueError as err:
        print("Error in calcularing RD, RAD of Giusti: {err}")



def calc_RAD_KIRCHLER_per_round(df):
    """ Calc Relative Absolute Deviation by Kirchler """
    if df is None:
        raise ValueError("No Dataframe for calculating average FV")
    if "price" not in df or "FV_KIRCHLER" not in df or "avg_FV_KIRCHLER" not in df:
        raise ValueError("Missing relevant variables")
    
    return round((abs(df['price'] - df['FV_KIRCHLER']) / abs(df['avg_FV_KIRCHLER'])),2)

def calc_RD_KIRCHLER_per_round(df):

    return round(((df['price'] - df['FV_KIRCHLER']) / abs(df['avg_FV_KIRCHLER'])),2)

def calc_deviation_KIRCHLER(df):
    """
    Main method for claculating deviations
    Categorizing rd values in overprcing, underpricing, fair
    Overpricing:  RD > 0
    Underpricing: RD < 0
    Fair:         RD =0  
    """
    if df is None:
        raise ValueError("No Dataframe for calculating average FV")

    #remove columns if existing
    df = df.drop(columns=[
        "avg_FV_KIRCHLER",
        "rd_Kirchler_per_round",
        "rad_Kirchler_per_round",
        "rd_Kirchler_categorization",
    ], errors="ignore")

    try: 
        df["avg_FV_KIRCHLER"] = df.groupby("session")["FV_KIRCHLER"].transform("mean")
        #df = pd.merge(df, f_avg_fv, on="session", how="left")
        df['rd_Kirchler_per_round'] = calc_RD_KIRCHLER_per_round(df)
        df['rad_Kirchler_per_round'] = calc_RAD_KIRCHLER_per_round(df)
        df['rd_Kirchler_categorization'] = np.where(df["rd_Kirchler_per_round"] > 0, "Overpricing",
                                np.where(df["rd_Kirchler_per_round"] < 0, "Underpricing", "Fair"))
        return df
    except ValueError as err:
        print(f"[Error] during the calculation of the RD, RAD of Kirchler:{err}")


def calc_RAD_Smith_per_round(df):
    """ RAD  by Smith """
    if df is None:
        raise ValueError("Missing df")
    if "price" not in df or "FV_SMITH" not in df:
        raise ValueError("Missing relevant variables")
        
    return round((abs(df['price'] - df['FV_SMITH']) / (df['FV_SMITH'])),2)

def calc_RD_Smith_per_round(df):
    """ RD per round by Smith """
    if df is None:
        raise ValueError("Missing df")

    if "price" not in df or "FV_SMITH" not in df:
        raise ValueError("Missing relevant variables")

    return round(((df['price'] - df['FV_SMITH']) / (df['FV_SMITH'])),2)

def calc_deviation_Smith(df):
    """
    Main method for claculating deviations
    Categorizing rd values in overprcing, underpricing, fair
    Overpricing:  RD > 0
    Underpricing: RD < 0
    Fair:         RD =0  
    """

    if df is None:
        raise ValueError("No Dataframe for calculating average FV")

    #remove columns if existing
    df = df.drop(columns=[
        "rd_Smith_per_round",
        "rad_Smith_per_round",
        "rd_Smith_categorization",
    ], errors="ignore")

    try:
        df['rd_Smith_per_round'] = calc_RAD_Smith_per_round(df)
        df['rad_Smith_per_round'] = calc_RD_Smith_per_round(df)
        df['rd_Smith_categorization'] = np.where(df["rd_Smith_per_round"] > 0, "Overpricing",
                                np.where(df["rd_Smith_per_round"] < 0, "Underpricing", "Fair"))
        return df
    except ValueError as error:
         print(f"[Error] during the calculation of the RD, RAD of Smith: {err}")

def calc_deviation(df_calc):
    """
    Main method for calculating the deviations of each researcher
    
    """
    try:
        col = ["session", "round", "price", "sell_count", "n_participants", "avg_trades",'remaining_rounds', 'FV_SMITH', 'FV_GUISTY','FV_KIRCHLER']
        df_calc_deviation = df_calc[col].drop_duplicates()
        print(df_calc_deviation.columns)
        df_calc_deviation = calc_deviation_Guisty(df_calc_deviation)
        df_calc_deviation = calc_deviation_KIRCHLER(df_calc_deviation)
        df_calc_deviation = calc_deviation_Smith(df_calc_deviation)
        df_calc = pd.merge(df_calc, df_calc_deviation, on= col, how= "left")
        return df_calc
    except ValueError as err:
        print(f"Error by calculating deviations: {err}")
    


df_calc = calc_deviation(df_calc.copy())
df_calc

Index(['session', 'round', 'price', 'sell_count', 'n_participants',
       'avg_trades', 'remaining_rounds', 'FV_SMITH', 'FV_GUISTY',
       'FV_KIRCHLER'],
      dtype='object')


,session,round,price,sell_count,n_participants,avg_trades,remaining_rounds,FV_SMITH,FV_GUISTY,FV_KIRCHLER,...,rd_GUISTY_per_round,rad_GUISTY_per_round,Category_Guisty_round,avg_FV_KIRCHLER,rd_Kirchler_per_round,rad_Kirchler_per_round,rd_Kirchler_categorization,rd_Smith_per_round,rad_Smith_per_round,rd_Smith_categorization
0,02-07-2024_14.15 (pre LSL),1,13.94,0,5,0.65,39,70.0,32.39,41.3,...,-0.77,0.77,Underpricing,27.65,-0.99,0.99,Underpricing,0.8,-0.8,Overpricing
1,02-07-2024_14.15 (pre LSL),1,13.94,0,5,0.65,39,70.0,32.39,41.3,...,-0.77,0.77,Underpricing,27.65,-0.99,0.99,Underpricing,0.8,-0.8,Overpricing
2,02-07-2024_14.15 (pre LSL),1,13.94,0,5,0.65,39,70.0,32.39,41.3,...,-0.77,0.77,Underpricing,27.65,-0.99,0.99,Underpricing,0.8,-0.8,Overpricing
3,02-07-2024_14.15 (pre LSL),1,13.94,0,5,0.65,39,70.0,32.39,41.3,...,-0.77,0.77,Underpricing,27.65,-0.99,0.99,Underpricing,0.8,-0.8,Overpricing
4,02-07-2024_14.15 (pre LSL),1,13.94,0,5,0.65,39,70.0,32.39,41.3,...,-0.77,0.77,Underpricing,27.65,-0.99,0.99,Underpricing,0.8,-0.8,Overpricing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444,29-05-2024_15.30 (pre LSL),40,14.27,0,6,0.95,0,70.0,14.55,14.0,...,-0.01,0.01,Underpricing,27.65,0.01,0.01,Overpricing,0.8,-0.8,Overpricing
3445,29-05-2024_15.30 (pre LSL),40,14.27,0,6,0.95,0,70.0,14.55,14.0,...,-0.01,0.01,Underpricing,27.65,0.01,0.01,Overpricing,0.8,-0.8,Overpricing
3446,29-05-2024_15.30 (pre LSL),40,14.27,0,6,0.95,0,70.0,14.55,14.0,...,-0.01,0.01,Underpricing,27.65,0.01,0.01,Overpricing,0.8,-0.8,Overpricing
3447,29-05-2024_15.30 (pre LSL),40,14.27,0,6,0.95,0,70.0,14.55,14.0,...,-0.01,0.01,Underpricing,27.65,0.01,0.01,Overpricing,0.8,-0.8,Overpricing


In [ ]:
def calculation_trade_price_minus_FV(df):
    """  calculating market price minus FV """

    if df is None:
        raise ValueError("No df")
    df = df.copy()

    df["price_minus_FV"] =  round(df['price'] - df["FV_KIRCHLER"],2)
    return df

df_calc = calculation_trade_price_minus_FV(df_calc)

## Creating Dataframe with all Fundamental Value Calculations  + Adding Categorizations

Categories based on price deviation of FV and decisions

In [16]:
# Creating a dataframe which is ready for the Statistical Test


df_ = df[["session", "participant", "round", "price", "transaction",'window_type', 'scl_mean', 'scl_std', 'scr_amplitude',
       'scr_auc', 'scr_count', 'scr_per_sec','signal_range', 'signal_std', "interest", "dividend" ]].copy()


# merging eda dataframe with FV dataframe
df_decision_FV = pd.merge(df_, df_calc.copy(), on=["session", "participant" ,"round", "price"], how="left")


#optimal_decisions_based_on_FV_Mispricing
df_decision_FV["Optimal_Decision_Guisty"] = np.where(df_decision_FV["rd_GUISTY_per_round"] > 0.3, "sell",
                 np.where(df_decision_FV["rd_GUISTY_per_round"] < -0.3, "buy", "None"))

#Categorize risk  based on FV of Giusty
df_decision_FV["Risk_Guisty"] = np.where(
    df_decision_FV["transaction"] == df_decision_FV["Optimal_Decision_Guisty"],
    0,
    np.where(
        (df_decision_FV["Optimal_Decision_Guisty"] == "None") & 
        (df_decision_FV["transaction"] != df_decision_FV["Optimal_Decision_Guisty"]),
        0.5,
        1
    )
)
df_decision_FV["Risk_Guisty"] = df_decision_FV["Risk_Guisty"].astype("category").cat.as_ordered()


#optimal_decisions_based_on_FV_Mispricing
df_decision_FV["Optimal_Decision_Kirchler"] = np.where(df_decision_FV["rd_Kirchler_per_round"] > 0.3, "sell",
                 np.where(df_decision_FV["rd_Kirchler_per_round"] < -0.3, "buy", "None"))

#Categorize risk  based on FV of Kirchler
df_decision_FV["Risk_Kirchler"] = np.where(
    df_decision_FV["transaction"] == df_decision_FV["Optimal_Decision_Kirchler"],
    0,
    np.where(
        (df_decision_FV["Optimal_Decision_Kirchler"] == "None") & 
        (df_decision_FV["transaction"] != df_decision_FV["Optimal_Decision_Kirchler"]),
        0.5,
        1
    )
)
df_decision_FV["Risk_Kirchler"] = df_decision_FV["Risk_Kirchler"].astype("category").cat.as_ordered()



#optimal_decisions_based_on_FV_Mispricing
df_decision_FV["Optimal_Decision_Smith"] = np.where(df_decision_FV["rd_Smith_per_round"] > 0.3, "sell",
                 np.where(df_decision_FV["rd_Smith_per_round"] < -0.3, "buy", "None"))

#Categorize risk  based on FV of Smith
df_decision_FV["Risk_Smith"] = np.where(
    df_decision_FV["transaction"] == df_decision_FV["Optimal_Decision_Smith"],
    0,
    np.where(
        (df_decision_FV["Optimal_Decision_Smith"] == "None") & 
        (df_decision_FV["transaction"] != df_decision_FV["Optimal_Decision_Smith"]),
        0.5,
        1
    )
)
df_decision_FV["Risk_Smith"] = df_decision_FV["Risk_Smith"].astype("category").cat.as_ordered()









#### Classify participant and session based on risk average


In [17]:
def risk_Categerory(df, column_group, market_mechnism):
    """  Calc the mean of risk decisision for each participant and each session"""

    if df is None:
        raise ValueError("No dataframe")

    var_col = ""
    if column_group == "session":
        var_col = "s"
    elif column_group == "participant":
        var_col = "p"

    #calculating the amount of categorues for column_group ("session" or "participant")
    risk_gu = (
        df
        .groupby([column_group, market_mechnism])
        .size()
        .unstack(fill_value=0)
        .reset_index()
        .rename(columns={
            0: f"{var_col}_Amount_{market_mechnism}_Category_0",
            0.5: f"{var_col}_Amount_{market_mechnism}_Category_0.5",
            1: f"{var_col}_Amount_{market_mechnism}_Category_1"
        })
    )

    risk_gu[f"{var_col}_{market_mechnism}_Total_Number"] = (
        risk_gu[[f"{var_col}_Amount_{market_mechnism}_Category_0", f"{var_col}_Amount_{market_mechnism}_Category_0.5", f"{var_col}_Amount_{market_mechnism}_Category_1"]]
        .sum(axis=1)
    )

    #average risk category
    risk_gu[f"{var_col}_Amount_{market_mechnism}_Average"] = (
        (risk_gu[f"{var_col}_Amount_{market_mechnism}_Category_0"] * 0 +
        risk_gu[f"{var_col}_Amount_{market_mechnism}_Category_0.5"] * 0.5 +
        risk_gu[f"{var_col}_Amount_{market_mechnism}_Category_1"] * 1)
        / risk_gu[f"{var_col}_{market_mechnism}_Total_Number"]
    ).round(2)

    #categorize in 3 similar size categories
    risk_gu[f"{var_col}_{market_mechnism}_categorization_average"] = pd.qcut(risk_gu[f"{var_col}_Amount_{market_mechnism}_Average"].rank(method="first"), q=3, labels=[0, 0.5, 1])
    return risk_gu

try:
    
    df_risk_Giusti_session = risk_Categerory(df_decision_FV.copy(), "session", "Risk_Guisty")
    df_risk_Giusti_participant = risk_Categerory(df_decision_FV.copy(), "participant", "Risk_Guisty")

    df_risk_Kirchler_session = risk_Categerory(df_decision_FV.copy(), "session", "Risk_Kirchler")
    df_risk_Kirchler_participant = risk_Categerory(df_decision_FV.copy(), "participant", "Risk_Kirchler")

    df_risk_Smith_session = risk_Categerory(df_decision_FV.copy(), "session", "Risk_Smith")
    df_risk_Smith_participant = risk_Categerory(df_decision_FV.copy(), "participant", "Risk_Smith")

    df_Risk_Mechanism_participant = pd.merge(df_risk_Giusti_participant, df_risk_Kirchler_participant, on= ["participant"], how = "inner")
    df_Risk_Mechanism_session = pd.merge(df_risk_Giusti_session, df_risk_Kirchler_session, on= ["session"], how = "inner")
    df_Risk_Mechanism_session


    df_Risk_participant = participants_demographics[["session", "participant"]]

    df_Risk_participant = pd.merge(df_Risk_participant, df_Risk_Mechanism_participant, on= "participant", how= "inner")
    df_Risk_participant_session = pd.merge(df_Risk_participant, df_Risk_Mechanism_session, on= "session", how= "inner")

    df_Risk_participant_session = pd.merge(df_Risk_participant_session, df_risk_Smith_participant, on= "participant", how= "inner")
    df_Risk_participant_session = pd.merge(df_Risk_participant_session, df_risk_Smith_session, on= "session", how= "inner")

    df_Risk_participant_session
except ValueError as err:
    print("Error by classify participant and session into risk classes")

In [18]:
# Merging classification Dataframe with Dataframe of FV 

df_FV_all = pd.merge(df_decision_FV.copy(),df_Risk_participant_session.copy(), on = ["session", "participant"], how = "left")
#save the dataframe
df_FV_all.to_csv(os.path.join(RESULTS, "df_FV_all.csv"), index=False)  

GENERAL_COLUMNS = ["session", "participant", "round", "price", "transaction", "cash", "stocks", "interest", "dividend"]
EDA_COLUMNS = ["window_type", "scl_mean", "scl_std", "scr_amplitude", "scr_auc", "scr_count", "scr_per_sec", "signal_range", "signal_std"]
AGGREGATES_COLUMNS = ["sell_counts", "n_participants", "avg_trades", "remaining_rounds", "n_stocks"]
CASH_ASSET_COLUMNS = ['cash_per_round', 'cash_asset_ratio_KIRCHLER', 'cash_asset_ratio_GUISTY', 'cash_asset_ratio_SMITH', 'mean_cash_asset_ratio_KIRCHLER', 'mean_cash_asset_ratio_SMITH','mean_cash_asset_ratio_GUISTY']
FV_RD_COLUMNS = ["FV_SMITH", "FV_GUISTY", "FV_KIRCHLER", "rd_Smith", "rad_Smith", "rd_Guisty", "rad_Guisty", "rd_Kirchler", "rad_Kirchler", "rd_Smith_per_round", "rad_Smith_per_round", "rd_GUISTY_per_round", "rad_GUISTY_per_round", "rd_Kirchler_per_round", "rad_Kirchler_per_round"]
RISK_FV_COLUMNS = ["Risk_Smith", "Risk_Guisty", "Risk_Kirchler", "p_Risk_Smith_categorization_average", "p_Risk_Guisty_categorization_average", "p_Risk_Kirchler_categorization_average","s_Risk_Smith_categorization_average", "s_Risk_Guisty_categorization_average", "s_Risk_Kirchler_categorization_average"]
JOIN_COLUMNS = GENERAL_COLUMNS + EDA_COLUMNS  + FV_RD_COLUMNS + RISK_FV_COLUMNS 

print(JOIN_COLUMNS)
REST_COLUMNS = list(set(df_FV_all.columns) - set(JOIN_COLUMNS))
REST_COLUMNS


df_decision_FV = df_FV_all[JOIN_COLUMNS]
df_decision_FV


['session', 'participant', 'round', 'price', 'transaction', 'cash', 'stocks', 'interest', 'dividend', 'window_type', 'scl_mean', 'scl_std', 'scr_amplitude', 'scr_auc', 'scr_count', 'scr_per_sec', 'signal_range', 'signal_std', 'FV_SMITH', 'FV_GUISTY', 'FV_KIRCHLER', 'rd_Smith', 'rad_Smith', 'rd_Guisty', 'rad_Guisty', 'rd_Kirchler', 'rad_Kirchler', 'rd_Smith_per_round', 'rad_Smith_per_round', 'rd_GUISTY_per_round', 'rad_GUISTY_per_round', 'rd_Kirchler_per_round', 'rad_Kirchler_per_round', 'Risk_Smith', 'Risk_Guisty', 'Risk_Kirchler', 'p_Risk_Smith_categorization_average', 'p_Risk_Guisty_categorization_average', 'p_Risk_Kirchler_categorization_average', 's_Risk_Smith_categorization_average', 's_Risk_Guisty_categorization_average', 's_Risk_Kirchler_categorization_average']


,session,participant,round,price,transaction,cash,stocks,interest,dividend,window_type,...,rad_Kirchler_per_round,Risk_Smith,Risk_Guisty,Risk_Kirchler,p_Risk_Smith_categorization_average,p_Risk_Guisty_categorization_average,p_Risk_Kirchler_categorization_average,s_Risk_Smith_categorization_average,s_Risk_Guisty_categorization_average,s_Risk_Kirchler_categorization_average
0,14_11_2024,0b8254y9,1,14.09,none,50.00,5,0.50,1.25,pre_decision,...,0.98,1.0,1.0,1.0,0.0,0.5,0.5,0.0,0.5,1.0
1,14_11_2024,0b8254y9,1,14.09,none,50.00,5,0.50,1.25,during_decision,...,0.98,1.0,1.0,1.0,0.0,0.5,0.5,0.0,0.5,1.0
2,14_11_2024,0b8254y9,1,14.09,none,50.00,5,0.50,1.25,NaN,...,0.98,1.0,1.0,1.0,0.0,0.5,0.5,0.0,0.5,1.0
3,14_11_2024,0b8254y9,1,14.09,none,50.00,5,0.50,1.25,during_feedback,...,0.98,1.0,1.0,1.0,0.0,0.5,0.5,0.0,0.5,1.0
4,14_11_2024,0b8254y9,1,14.09,none,50.00,5,0.50,1.25,post_feedback,...,0.98,1.0,1.0,1.0,0.0,0.5,0.5,0.0,0.5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15692,29-05-2024_15.30 (pre LSL),k9j9nh9y,36,15.70,none,242.63,2,2.43,2.50,during_feedback,...,0.04,1.0,0.5,0.5,0.5,0.0,0.0,0.5,0.0,0.0
15693,29-05-2024_15.30 (pre LSL),k9j9nh9y,37,15.12,none,247.56,2,2.48,2.50,during_feedback,...,0.04,1.0,0.5,0.5,0.5,0.0,0.0,0.5,0.0,0.0
15694,29-05-2024_15.30 (pre LSL),k9j9nh9y,38,14.87,none,252.54,2,2.53,1.60,during_feedback,...,0.02,1.0,0.5,0.5,0.5,0.0,0.0,0.5,0.0,0.0
15695,29-05-2024_15.30 (pre LSL),k9j9nh9y,39,13.97,none,256.67,2,2.57,2.50,during_feedback,...,0.03,1.0,0.5,0.5,0.5,0.0,0.0,0.5,0.0,0.0


## Volatility - Measures

In this section I will calculate the volatility measures by Ederingtion and Kaufman.

#### Change-Price_Fluctuations (n= 5rounds)

In [ ]:
 
def change_price_fluctations_5_rounds(df):
    """ Calculating price the change of price between 5 rounds"""
    if df is None:
        raise ValueError("No df")
    if "price" not in df or  "price_5_before" not in df:
        raise ValueError("Missing important variables")

    df['change_price'] = round(df['price'] - df['price_5_before'],2)
    return df
    
try:
    df_volatility = df[['session', 'round', 'price']].copy()
    df_volatility = df_volatility.drop_duplicates()

    df_volatility['price_5_before'] = (df_volatility.groupby('session')['price'].shift(5))    
    df_volatility = change_price_fluctations_5_rounds(df_volatility)
except ValueError as err:
    print(f"[Error] by calculating price changes during 5 periods {err}")


#### Maximum price Fluctuations during the n-days: (n= 5, days = round)

In [ ]:
  
def max_min_price_5_rounds(df_volatility):
  """ Calculating the difference between the max and min prince during a trading window of 5 rounds"""
  if df_volatility is None:
    raise ValueError("Missing dataframe")

  df_volatility.copy()
  df_volatility = df_volatility.sort_values(['session', 'round']).drop_duplicates()

  #min volatilty measure of the window
  df_volatility['min_5_including'] = (
      df_volatility
        .groupby('session')['price']
        .transform(lambda s: s.rolling(window=5, min_periods=1).min())
  )

   #max volatilty measure of the window
  df_volatility['max_5_including'] = (
      df_volatility
        .groupby('session')['price']
        .transform(lambda s: s.rolling(window=5, min_periods=1).max())
  )

  #subtract
  df_volatility['deviation_max_min'] = round(df_volatility['max_5_including'] - df_volatility['min_5_including'],2)

  return df_volatility

df_volatility = max_min_price_5_rounds(df_volatility)

#### Sum of the absolute price changes over n-days (n= 5)

In [ ]:
def sum_absolute_price_change(df_volatility, n = 5):
  """ 
  Calculating the sum of the absolute deviation within the winodw (length = 5)
  """ 
  if df_volatility is None:
    raise ValueError("Missing df")

  #calculating the difference of the price t-1 and t
  df_volatility["abs_diff"] = (
      df_volatility.groupby("session")["price"].diff().abs().round(2))

  # Sum of last n  rounrds (inkl. actual round Runde)
  # Calculating the sum of the absolute deviation within the winodw (length = 5)
  df_volatility[f"sum_abs_diff_{n}"] = (
      df_volatility.groupby("session")["abs_diff"]
        .transform(lambda s: s.rolling(window=n, min_periods=1).sum()).round(2)
  )

  return df_volatility


df_volatility = sum_absolute_price_change(df_volatility.copy())

#### Standard Deviation by Ederington (2006)

In [ ]:
# standard deviation by Ederington (2006)
# actual dividend, not expected dividend
def average_dividend_per_share_per_session(df, expected_D = 0.07):
    if df is None:
        raise ValueError("No dataframe available")

    dividend = (
        df[['session', 'participant', 'round', 'stocks', 'dividend']]
        .drop_duplicates()
        .sort_values(['session', 'participant', 'round'])
        .copy()
    )

    
    # actual dividend per stock; if stocks>0, inf -> NaN
    dividend['dividend_per_share'] = np.where(
        (dividend['stocks'] > 0) & np.isfinite(dividend['stocks']),
        dividend['dividend'] / dividend['stocks'],
        np.nan
    ).round(2)
    dividend['dividend_per_share'].replace([np.inf, -np.inf], np.nan, inplace=True)

    # Mean per round (only valid participants), fill empty rounds with 0.7 
    dividend_mean = (
        dividend.groupby(['session', 'round'], as_index=False)['dividend_per_share']
                .agg(D_t_obs='mean')
    ).round(2)
    dividend_mean['D_t'] = dividend_mean['D_t_obs'].fillna(expected_D)
    dividend_mean.drop(columns=['D_t_obs'], inplace=True)

    return df.merge(dividend_mean, on=['session', 'round'], how='left')
    

def calc_volatility_by_Ederingtion(df, n =5):
    """
    Main method: calculate volatility (standard deviation) by Ederington
    """
    
    df_volatility = df[['session','round', 'price', 'D_t']].drop_duplicates().sort_values(['session', 'round'])
    for j in range(1,n+1):
        #caclculating the difference between actual price and price j rounds before
        if j == n+1: 
            df_volatility[f'price_t_minus_{j}'] = (df_volatility.groupby('session')['price'].shift(j).round(2)) 
        else: 
            df_volatility[f'price_t_minus_{j}'] = (df_volatility.groupby('session')['price'].shift(j).round(2)) 
            df_volatility[f'dividend_t_minus_{j}'] = (df_volatility.groupby('session')['D_t'].shift(j).round(2)) 
    rows_all_session = []
    #calculating MAD and AMAD
    for session, group in df_volatility.groupby('session'):
        rows_all_session = volatility_mad_amad(group,rows_all_session)
    df_volatility_rows = pd.DataFrame(rows_all_session)
    #merge results with actual frame
    df_volatility = pd.merge(df_volatility, df_volatility_rows, how="left", on=["session",'round', 'price', 'D_t'])
    return df_volatility

def volatility_mad_amad(group, rows_all_session, n= 5):
    """Calculating MAD and AMAD by Ederington"""

    if group is None:
        raise ValueError("No dataframe available")
    
    for idx, line in group.iterrows():
        session = line['session']
        round_ = line['round']
        price_t = line['price'] 
        sum_daily_returns_Vol = 0
        sum_daily_return_MAD = 0
        window_returns = []

        for j in range(0,n):
            if j == 0:
                price_t_minus_j = line['price']
                price_t_minus_j_minus_1 = line[f'price_t_minus_1']
                dividend_t_minus_j = line['D_t']
            else:
                price_t_minus_j = line[f'price_t_minus_{j}']
                price_t_minus_j_minus_1 = line[f'price_t_minus_{j+1}']
                dividend_t_minus_j = line[f'dividend_t_minus_{j}']
            
            daily_retun_t_minus_j = math.log((price_t_minus_j + dividend_t_minus_j) / price_t_minus_j_minus_1)
            window_returns.append(daily_retun_t_minus_j)

            R_t_minus_j= (price_t_minus_j + dividend_t_minus_j)/price_t_minus_j_minus_1
            daily_returns_t_minus_j = math.log(R_t_minus_j)
            mean_return = float(np.mean(window_returns))
            r_t_minus_j = daily_returns_t_minus_j - mean_return
            sum_daily_returns_Vol += (r_t_minus_j)**2
            sum_daily_return_MAD += abs(r_t_minus_j)   

        #calc V_t, MAD_t, AMAD_t for actual round t
        V_t = round(math.sqrt((1/n) * sum_daily_returns_Vol),2)
        MAD_t =round(((1/n) * sum_daily_return_MAD),2)
        AMAD_t = round((math.sqrt(math.pi/2) * MAD_t),2)

        row = {'session': session,'round':round_,  'price': price_t,'D_t':line['D_t'], 'V_t': V_t , 'MAD_t': MAD_t,'AMAD_t': AMAD_t, "mean_return": mean_return }
        rows_all_session.append(row)
        sum_daily_returns_Vol = 0
        sum_daily_return_MAD = 0
    
    return rows_all_session



new_df = average_dividend_per_share_per_session(df.copy())
df_volatility_Ederington = calc_volatility_by_Ederingtion(new_df)
df_volatility_Ederington

,session,round,price,D_t,price_t_minus_1,dividend_t_minus_1,price_t_minus_2,dividend_t_minus_2,price_t_minus_3,dividend_t_minus_3,price_t_minus_4,dividend_t_minus_4,price_t_minus_5,dividend_t_minus_5,V_t,MAD_t,AMAD_t,mean_return
0,02-07-2024_14.15 (pre LSL),1,13.94,0.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,02-07-2024_14.15 (pre LSL),2,14.76,1.01,13.94,0.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02-07-2024_14.15 (pre LSL),3,15.65,0.72,14.76,1.01,13.94,0.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,02-07-2024_14.15 (pre LSL),4,15.46,0.57,15.65,0.72,14.76,1.01,13.94,0.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,02-07-2024_14.15 (pre LSL),5,16.73,0.87,15.46,0.57,15.65,0.72,14.76,1.01,13.94,0.66,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,29-05-2024_15.30 (pre LSL),36,15.70,0.42,16.77,0.62,15.63,0.68,17.29,0.76,18.63,0.86,17.13,0.86,0.07,0.05,0.06,0.021253
836,29-05-2024_15.30 (pre LSL),37,15.12,0.75,15.70,0.42,16.77,0.62,15.63,0.68,17.29,0.76,18.63,0.86,0.05,0.04,0.05,-0.002407
837,29-05-2024_15.30 (pre LSL),38,14.87,0.78,15.12,0.75,15.70,0.42,16.77,0.62,15.63,0.68,17.29,0.76,0.05,0.04,0.05,0.010809
838,29-05-2024_15.30 (pre LSL),39,13.97,0.77,14.87,0.78,15.12,0.75,15.70,0.42,16.77,0.62,15.63,0.68,0.04,0.03,0.04,0.020723


In [23]:
# check returns:
df_volatility_Ederington["return"] = np.log((df_volatility_Ederington["price"] + df_volatility_Ederington["dividend_t_minus_1"])/df_volatility_Ederington["price_t_minus_1"])

df_volatility_Ederington["return"]

x = df_volatility_Ederington.groupby("session")["return"].median()
x


session
02-07-2024_14.15 (pre LSL)    0.052756
02-07-2024_16.30 (pre LSL)    0.044164
03-07-2024_15.15 (pre LSL)    0.052928
06-08-2024_13.15              0.021431
07_11_2024                    0.038160
12_11_2024                    0.056760
13-08-2024_13.45 (No fMRI)    0.058516
13_11_2024                    0.055470
14-08-2024_15.15              0.029917
14_11_2024                    0.071224
19_11_2024                    0.045940
20-08-2024_13.45(no fMRI)     0.059208
20-08-2024_16.15(no fMRI)     0.060254
21_11_2024 (no fMRI)          0.049325
27-08-2024_13.45              0.074331
27_11_2024                    0.053423
28-05-2024_14.30 (pre LSL)    0.048566
28-05-2024_17.15 (pre LSL)    0.054531
28-08-2024_15.15              0.019285
28_11_2024                    0.040955
29-05-2024_15.30 (pre LSL)    0.021250
Name: return, dtype: float64

In [ ]:
#check nans
nan_summary = (
    df_volatility_Ederington
    .assign(has_nan = df_volatility_Ederington[['V_t','MAD_t','AMAD_t']].isna().any(axis=1))
    .groupby('session')['has_nan']
    .value_counts()
    .unstack(fill_value=0)
    .rename(columns={True:'NaN_rows', False:'Valid_rows'})
)

print(nan_summary)

has_nan                     Valid_rows  NaN_rows
session                                         
02-07-2024_14.15 (pre LSL)          35         5
02-07-2024_16.30 (pre LSL)          35         5
03-07-2024_15.15 (pre LSL)          35         5
06-08-2024_13.15                    35         5
07_11_2024                          35         5
12_11_2024                          35         5
13-08-2024_13.45 (No fMRI)          35         5
13_11_2024                          35         5
14-08-2024_15.15                    35         5
14_11_2024                          35         5
19_11_2024                          35         5
20-08-2024_13.45(no fMRI)           35         5
20-08-2024_16.15(no fMRI)           35         5
21_11_2024 (no fMRI)                35         5
27-08-2024_13.45                    35         5
27_11_2024                          35         5
28-05-2024_14.30 (pre LSL)          35         5
28-05-2024_17.15 (pre LSL)          35         5
28-08-2024_15.15    

In [ ]:
def categorization_Risk_Volatility(df_vol_full):
    """Categorization (global, session) of Risk based on terciles  """
    if df_vol_full is None:
        raise ValueError("DataFrame is missing")

    #drop duplicates and remove dropna values
    df_vol = df_vol_full[["session", "round", "V_t"]].drop_duplicates().dropna()


    #Classify for for all volatility values into 3 groups
    df_vol['Vol_Classification_Global'] = pd.qcut(df_vol["V_t"], q=3, labels=[0, 0.5, 1])
    df_vol["Vol_Classification_Global"] = df_vol["Vol_Classification_Global"].astype("category").cat.as_ordered()

    #Classify volatility values within each session  into 3  groups
    df_vol["Vol_Classification_intra_session"] = df_vol.groupby("session")["V_t"].transform(
    lambda s: pd.qcut(s.rank(method="first"), q=3, labels=[0, 0.5, 1]))
    df_vol["Vol_Classification_intra_session"] =  df_vol["Vol_Classification_intra_session"].astype("category").cat.as_ordered()
    
    #Calc the mean voaltility of each session
    session_vol_mean = (df_vol.groupby("session",as_index=False)["V_t"].mean().round(2).rename(columns={"V_t": "Mean_V_t"}))
    # Categorize session mean volatility into 3  groups
    session_vol_mean["Vol_Classification_between_session"] = (pd.qcut(session_vol_mean["Mean_V_t"], q=3, labels=[0, 0.5, 1]))
    session_vol_mean["Vol_Classification_between_session"] = session_vol_mean["Vol_Classification_between_session"].astype("category").cat.as_ordered()

    ## Merging 
    df_vol = df_vol.merge(session_vol_mean[["session", "Mean_V_t", "Vol_Classification_between_session"]],
        on="session", how="left", validate="many_to_one"
    )

    df_voll_full = pd.merge(df_vol_full, df_vol, on = ["session", "round", "V_t"], how= "left")

    return df_voll_full

try: 
    df_volatility_Ederington = categorization_Risk_Volatility(df_volatility_Ederington.copy())

    #Number of values per categories 

    print(df_volatility_Ederington.Vol_Classification_Global.value_counts(0))
    print("\n")
    print(df_volatility_Ederington.Vol_Classification_intra_session.value_counts(0))
    print("\n")
    print(df_volatility_Ederington.Vol_Classification_between_session.value_counts(0))

    df_volatility_meausures = df_volatility_Ederington.merge(df_volatility, on=['session', 'round', 'price'], how='left')
    df_volatility_meausures.head(40)

    #Merging EDA Variables with df volatility measures
    COL = GENERAL_COLUMNS + EDA_COLUMNS
    partial_df = df[COL]
    df_vol = partial_df.merge(df_volatility_meausures, on = ["session", "round", "price"], how= 'left')


    #save Volatility as csv file
    print(f"Saving Volatility measures to csv: {os.path.join(RESULTS, "df_volatility.csv")}")
    df_vol.to_csv(os.path.join(RESULTS, "df_volatility.csv"), index=False)
    print("✅ Done saving")

    VOL_COLUMN = ["V_t", "MAD_t", "AMAD_t"]
    VOL_RISK_CLASSIFICATION_COLUMN = ["Vol_Classification_Global", "Vol_Classification_intra_session", "Vol_Classification_between_session"]
    VOL_RISK_COLUMN = VOL_COLUMN + VOL_RISK_CLASSIFICATION_COLUMN 


    df_volatility_add_to_all = df_vol[["session", "round"] + VOL_RISK_COLUMN].drop_duplicates()

  
    # Merging FV dataframe with volatility -> Analysis ready DataFrame
    df_analysis_ready = pd.merge(df_decision_FV,df_volatility_add_to_all, on = ["session", "round"], how = "left" )
    df_analysis_ready = pd.merge(df_analysis_ready, participants_demographics, on = ["session", "participant"], how = "left")

    
    # saving the categorcal datatypes 
    schema = {}
    print(f"Saving schema to csv: {os.path.join(RESULTS, "schema_results.json")}")
    for c in df_analysis_ready.columns:
        if pd.api.types.is_categorical_dtype(df_analysis_ready[c]):
            schema[c] = {
                "categories": df_analysis_ready[c].cat.categories.tolist(),
                "ordered": bool(df_analysis_ready[c].cat.ordered),
            }
    with open(os.path.join(RESULTS, "schema_results.json"), "w") as f:
        json.dump(schema, f)
    print("✅ Done saving")

    #save df_analysis_ready to csv
    print(f"Saving df_analysis_ready to csv: {os.path.join(RESULTS, "statistical_analysis_ready.csv")}")
    df_analysis_ready.to_csv(os.path.join(RESULTS, "statistical_analysis_ready.csv"), index=False)  
    print("✅ Done saving")
except ValueError as err:
    print("Error in calculating, merging and saving results", err)



Vol_Classification_Global
0.0    339
0.5    229
1.0    167
Name: count, dtype: int64


Vol_Classification_intra_session
0.0    252
1.0    252
0.5    231
Name: count, dtype: int64


Vol_Classification_between_session
0.5    315
0.0    245
1.0    175
Name: count, dtype: int64
Saving Volatility measures to csv: /Users/johannanagel/Masterarbeit_Programming/output_postfeedback/Main_Part/DataFrames/df_volatility.csv
✅ Done saving
Saving schema to csv: /Users/johannanagel/Masterarbeit_Programming/output_postfeedback/Main_Part/DataFrames/schema_results.json
✅ Done saving
Saving df_analysis_ready to csv: /Users/johannanagel/Masterarbeit_Programming/output_postfeedback/Main_Part/DataFrames/statistical_analysis_ready.csv
✅ Done saving


In [26]:
df_analysis_ready.columns

Index(['session', 'participant', 'round', 'price', 'transaction', 'cash',
       'stocks', 'interest', 'dividend', 'window_type', 'scl_mean', 'scl_std',
       'scr_amplitude', 'scr_auc', 'scr_count', 'scr_per_sec', 'signal_range',
       'signal_std', 'FV_SMITH', 'FV_GUISTY', 'FV_KIRCHLER', 'rd_Smith',
       'rad_Smith', 'rd_Guisty', 'rad_Guisty', 'rd_Kirchler', 'rad_Kirchler',
       'rd_Smith_per_round', 'rad_Smith_per_round', 'rd_GUISTY_per_round',
       'rad_GUISTY_per_round', 'rd_Kirchler_per_round',
       'rad_Kirchler_per_round', 'Risk_Smith', 'Risk_Guisty', 'Risk_Kirchler',
       'p_Risk_Smith_categorization_average',
       'p_Risk_Guisty_categorization_average',
       'p_Risk_Kirchler_categorization_average',
       's_Risk_Smith_categorization_average',
       's_Risk_Guisty_categorization_average',
       's_Risk_Kirchler_categorization_average', 'V_t', 'MAD_t', 'AMAD_t',
       'Vol_Classification_Global', 'Vol_Classification_intra_session',
       'Vol_Classificatio

In [27]:
#Merging all data together for visualization
df_vol_all = pd.merge(df_volatility_Ederington, df_volatility, on = ["session", "round", "price"], how= "left")
df_vol_all = pd.merge(df_FV_all, df_vol_all, on= ["session", "round", "price"], how = 'left')
df_vol_all = pd.merge(df_vol_all, participants_demographics, on= ["session","participant"], how = 'left' )

#check nan values
df_vol_all.isna().sum()[df_vol_all.isna().sum()>0]




window_type                           3352
scr_amplitude                         4475
price_t_minus_1                        411
dividend_t_minus_1                     411
price_t_minus_2                        822
dividend_t_minus_2                     822
price_t_minus_3                       1232
dividend_t_minus_3                    1232
price_t_minus_4                       1640
dividend_t_minus_4                    1640
price_t_minus_5                       2045
dividend_t_minus_5                    2045
V_t                                   2045
MAD_t                                 2045
AMAD_t                                2045
mean_return                           2045
return                                 411
Vol_Classification_Global             2045
Vol_Classification_intra_session      2045
Mean_V_t                              2045
Vol_Classification_between_session    2045
price_5_before                        2051
change_price                          2051
abs_diff   

In [28]:
#save df_analysis_ready to csv
print(f"Saving df_analysis_ready to csv: {os.path.join(RESULTS, "all_data_combined.csv")}")
df_vol_all.to_csv(os.path.join(RESULTS, "all_data_combined.csv"), index=False)  
print("✅ Done saving")

Saving df_analysis_ready to csv: /Users/johannanagel/Masterarbeit_Programming/output_postfeedback/Main_Part/DataFrames/all_data_combined.csv
✅ Done saving
